# jieba

## 主要功能:
***
### 1. 分词

* jieba.cut 方法接受三个输入参数: 需要分词的字符串；cut_all 参数用来控制是否采用全模式；HMM 参数用来控制是否使用 HMM 模型
* jieba.cut_for_search 方法接受两个参数：需要分词的字符串；是否使用 HMM 模型。该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细待分词的字符串可以是 unicode 或 UTF-8 字符串、GBK 字符串。注意：不建议直接输入 GBK 字符串，可能无法预料地错误解码成 UTF-8
* jieba.cut 以及 jieba.cut_for_search 返回的结构都是一个可迭代的 generator，可以使用 for 循环来获得分词后得到的每一个词语(unicode)，或者用
* jieba.lcut 以及 jieba.lcut_for_search 直接返回 list
* jieba.Tokenizer(dictionary=DEFAULT_DICT) 新建自定义分词器，可用于同时使用不同词典。jieba.dt 为默认分词器，所有全局分词相关函数都是该分词器的映射。

In [1]:
# encoding = UTF-8
import jieba
import jieba.posseg as pseg
from __future__ import unicode_literals
import sys,os
from whoosh.index import create_in,open_dir
from whoosh.fields import *
from whoosh.qparser import QueryParser
from jieba.analyse import ChineseAnalyzer

In [26]:
seg_list = jieba.cut('我来到北京清华大学',cut_all=True)
print('Full Mode:' + '/'.join(seg_list)) #全模式

seg_list = jieba.cut('我来到北京清华大学',cut_all=False)
print('Default Mode:' + '/'.join(seg_list)) # 精确模式

seg_list = jieba.cut('我来到北京清华大学')
print(', '.join(seg_list)) # 默认为精确模式

seg_list = jieba.cut_for_search('小明硕士毕业于中国科学院计算所，后在日本京都大学深造') # 搜索引擎模式
print('Search Engin Mode'+', '.join(seg_list))

Full Mode:我/来到/北京/清华/清华大学/华大/大学
Default Mode:我/来到/北京/清华大学
我, 来到, 北京, 清华大学
Search Engin Mode小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


### 2. 添加自定义词典
***
**载入词典**
* 开发者可以指定自己自定义的词典，以便包含 jieba 词库里没有的词。虽然 jieba 有新词识别能力，但是自行添加新词可以保证更高的正确率
* 用法： <b>jieba.load_userdict(file_name)</b> # file_name 为文件类对象或自定义词典的路径
* 词典格式和 dict.txt 一样，一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。
* 词频省略时使用自动计算的能保证分出该词的词频。
* 更改分词器（默认为 jieba.dt）的 tmp_dir 和 cache_file 属性，可分别指定缓存文件所在的文件夹及其文件名，用于受限的文件系统。


In [55]:
print('/'.join(jieba.cut('李小福是创新办主任也是云计算方面的专家')))

李小福/是/创新办/主任/也/是/云计算/方面/的/专家


In [56]:
jieba.add_word('创新办')
jieba.add_word('云计算')
print('/'.join(jieba.cut('李小福是创新办主任也是云计算方面的专家')))

李小福/是/创新办/主任/也/是/云计算/方面/的/专家


**调整词典**
* 使用 <b>add_word(word, freq=None, tag=None) </b>和 <b>del_word(word) </b>可在程序中动态修改词典。

* 使用 <b>suggest_freq(segment, tune=True)</b> 可调节单个词语的词频，使其能（或不能）被分出来。

* 注意：自动计算的词频在使用 HMM 新词发现功能时可能无效。

In [57]:
print('/'.join(jieba.cut('如果放到post中将出错。',HMM = False)))
jieba.suggest_freq(('中','将'), tune=True)
print('/'.join(jieba.cut('如果放到post中将出错。',HMM = False)))
print('/'.join(jieba.cut('「台中」正确应该不会被切开',HMM = False)))
jieba.suggest_freq('台中',tune=True)
print('/'.join(jieba.cut('「台中」正确应该不会被切开',HMM = False)))

如果/放到/post/中/将/出错/。
如果/放到/post/中/将/出错/。
「/台中/」/正确/应该/不会/被/切开
「/台中/」/正确/应该/不会/被/切开


### 3. 关键词抽取

### 基于 TF-IDF 算法的关键词抽取

> **import jieba.analyse**

** jieba.analyse.extract_tags(sentence, topK = 20, withWeight = False, allowPOS = ())**

**参数说明**
* sentence 为待提取文本
* topK 为返回几个TF/IDF 权重最大的关键词,默认值为20
* withWeight 为是否一并返回关键词的权重值,默认为False
* allowPOS仅包括制定词性的词, 默认为空,既不筛选

---

** jieba.analyse.TFIDF(idf_path=None) **

* 新建 TFIDF 实例，idf_path 为 IDF 频率文件
* IDF 逆向文件词频

**1. 关键词提取使用逆向文件频率(IDF)文本语料库可以切换成自定义语料库的路径**

> <b >用法</b>: jieba.analyse.set_idf_path(file_name)  #file_name为自定义语料库的路径

**2. 关键词提取所使用的停用词(stop words)文本语料库可以切换成自定义语料库的路径**

> <b>用法:</b> jieba.analyse.set_stop_words(file_name) # file_name 为自定义语料库的路径

**3. 关键词也可以一并返回关键词权重值**

---

### 基于TextRank算法的关键词抽取

<b>jieba.analys.textrank(sentence, topK = 20, withWeight = False, allowPos = ("ns","n","vn","v")) </b> 直接使用, 接口相同, 之一默认过滤词性

<b>jieba.analyse.TextRank()</b>新建自定义TextRank实例
> <b>基本思想:</b>
 1. 将待抽取关键词的文本进行分词
 2. 以固定窗口大小(默认为5, 通过span属性调整), 词之间的共现关系, 构建图
 3. 计算图中节点的pagerank, 注意是**无向带权图**

In [40]:
# example for tags extract
import jieba.analyse
content = '''我沒有心
我沒有真實的自我
我只有消瘦的臉孔
所謂軟弱
所謂的順從一向是我
的座右銘

而我
沒有那海洋的寬闊
我只要熱情的撫摸
所謂空洞
所謂不安全感是我
的墓誌銘

而你
是否和我一般怯懦
是否和我一般矯作
和我一般囉唆

而你
是否和我一般退縮
是否和我一般肌迫
一般地困惑

我沒有力
我沒有滿腔的熱火
我只有滿肚的如果
所謂勇氣
所謂的認同感是我
隨便說說

而你
是否和我一般怯懦
是否和我一般矯作
是否對你來說
只是一場遊戲
雖然沒有把握

而你
是否和我一般退縮
是否和我一般肌迫
是否對你來說
只是逼不得已
雖然沒有藉口'''

tags = jieba.analyse.extract_tags(content, topK=10)
print(";".join(tags))

所謂;是否;一般;沒有;矯作;退縮;肌迫;雖然;怯懦;真實


In [43]:
# IDF 逆向文件频率文本语料库切换成自定义语料库
jieba.analyse.set_idf_path('idf.txt.big')
tags = jieba.analyse.extract_tags(content, topK=10)
print("; ".join(tags))

一般; 所謂; 是否; 只有; 怯懦; 矯作; 退縮; 肌迫; 只是; 有心


In [47]:
# stop words 停用词文本语料库切换成自定义的语料库
jieba.analyse.set_stop_words('stop_words.txt')
tags = jieba.analyse.extract_tags(content, topK=10)
print(";".join(tags))

所謂;只有;怯懦;矯作;退縮;肌迫;有心;真實;自我;消瘦


In [53]:
# 返回关键词以及权重
tags = jieba.analyse.extract_tags(content, topK=10, withWeight=True)
for tag in tags:
    print(("tag: %s\tweight: %f")%(tag[0],tag[1]))

tag: 所謂	weight: 1.526141
tag: 只有	weight: 0.508714
tag: 怯懦	weight: 0.508714
tag: 矯作	weight: 0.508714
tag: 退縮	weight: 0.508714
tag: 肌迫	weight: 0.508714
tag: 有心	weight: 0.254357
tag: 真實	weight: 0.254357
tag: 自我	weight: 0.254357
tag: 消瘦	weight: 0.254357


In [54]:
# 基于 TextRank 算法的关键词抽取的示例


### 4. 词性标注
---
> 1. jieba.posseg.POSTokenizer(tokenizer = None)
>> 新建自定义分词器, tokenizer参数可指定内部使用的 jieba.Tokenizer分词器. jieba.posseg.dt为默认词性标注分词器

> 2. 标注句子分词后每个词的词性, 采用和ictclas 兼容的标记法

> 3. 用法如下:


In [67]:
?jieba.Tokenizer

In [2]:
import jieba.posseg as pseg
words = pseg.cut('我爱北京天安门')
for w, f in words:
    print(("word: %s\tflag: %s")%(w,f))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\JANNEY~1.ZHA\AppData\Local\Temp\jieba.cache
Loading model cost 0.758 seconds.
Prefix dict has been built succesfully.


word: 我	flag: r
word: 爱	flag: v
word: 北京	flag: ns
word: 天安门	flag: ns


### 5. 并行分词
 ---
 
 * <b>原理:</b>将目标文本按行分割后, 把各行文本分配到多个python 进行并行分词, 然后归并结果, 从而获得分词速度的客观提升
 * 基于python 自带的<b>multiprocessing</b>模块, 目前暂不支持windows...
 * <b>用法:</b>
 
 >> <b>jieba.enable_parallel(4)</b> # 开启并行分词模式, 参数为并行进程数
 >> <b>jieba.disable_parallel()</b> # 关闭并行分词模式
 
 * 实验结果: 在 4 核 3.4GHz Linux 机器上，对金庸全集进行精确分词，获得了 1MB/s 的速度，是单进程版的 3.3 倍。
 
 
 **注意** 并行分词仅支持默认分词器 jieba.dt 和jieba.posseg.dt
 
 ---
 
### 6. Tokenizer: 返回词语在原文的起止位置
---
**注意**, 输入参数只接受**unicode**


In [4]:
# 默认模式
result = jieba.tokenize(u'永和有限服装饰品永和有限公司')
for tk in result:
    print("word %s \t start: %d \t end: %d"%(tk[0],tk[1],tk[2]))

word 永和 	 start: 0 	 end: 2
word 有限 	 start: 2 	 end: 4
word 服装 	 start: 4 	 end: 6
word 饰品 	 start: 6 	 end: 8
word 永和 	 start: 8 	 end: 10
word 有限公司 	 start: 10 	 end: 14


In [61]:
# 搜索模式
result = jieba.tokenize(u'永和服装饰品有限公司', mode = "search")
for tk in result:
    print("word %s \t start: %d \t end: %d"%(tk[0],tk[1],tk[2]))

word 永和 	 start: 0 	 end: 2
word 服装 	 start: 2 	 end: 4
word 饰品 	 start: 4 	 end: 6
word 有限 	 start: 6 	 end: 8
word 公司 	 start: 8 	 end: 10
word 有限公司 	 start: 6 	 end: 10


### 7. ChineseAnalyzer for Whoosh 搜索引擎
---
**引用: ** from jieba.analyse import ChineseAnalyzer

In [5]:
from __future__ import unicode_literals
import sys,os
from whoosh.index import create_in,open_dir
from whoosh.fields import *
from whoosh.qparser import QueryParser
from jieba.analyse import ChineseAnalyzer

In [6]:
analyzer = ChineseAnalyzer()
schema = Schema(title = TEXT(stored = True), path = ID(stored = True), content = TEXT(stored = True, analyzer = analyzer))


In [7]:
if not os.path.exists("tmp"):
    os.mkdir("tmp")


In [8]:
ix = open_dir("tmp")

In [9]:
searcher = ix.searcher()
parser = QueryParser("content", schema=ix.schema)

In [10]:
for keyword in ("水果世博园","你","first","中文","交换机","交换"):
    print("result of ",keyword)
    q = parser.parse(keyword)
    results = searcher.search(q)
    #print(results)
    for hit in results:
        print(hit)
        print(hit.highlights('content'))
    print("="*10)

result of  水果世博园
<Hit {'content': '买水果然后来世博园。', 'path': '/c', 'title': 'document3'}>
买<b class="match term0">水果</b>然后来<b class="match term1">世博园</b>
result of  你
<Hit {'content': 'The second one 你 中文测试中文 is even more interesting! 吃水果', 'path': '/b', 'title': 'document2'}>
second one <b class="match term0">你</b> 中文测试中文 is even more interesting
result of  first
<Hit {'content': 'This is the first document we’ve added!', 'path': '/a', 'title': 'document1'}>
<b class="match term0">first</b> document we’ve added
result of  中文
<Hit {'content': 'The second one 你 中文测试中文 is even more interesting! 吃水果', 'path': '/b', 'title': 'document2'}>
second one 你 <b class="match term0">中文</b>测试<b class="match term0">中文</b> is even more interesting
result of  交换机
<Hit {'content': '工信处女干事每月经过下属科室都要亲口交代24口交换机等技术性器件的安装工作', 'path': '/c', 'title': 'document4'}>
干事每月经过下属科室都要亲口交代24口<b class="match term0">交换机</b>等技术性器件的安装工作
result of  交换
<Hit {'content': '咱俩交换一下吧。', 'path': '/c', 'title': 'document4'}>
咱俩<b class="

result of  水果世博园

买<b class="match term0">水果</b>然后来<b class="match term1">世博园</b>
***
result of  你

second one <b class="match term0">你</b> 中文测试中文 is even more interesting
***
result of  first

<b class="match term0">first</b> document we’ve added
***
result of  中文

second one 你 <b class="match term0">中文</b>测试<b class="match term0">中文</b> is even more interesting
***
result of  交换机

干事每月经过下属科室都要亲口交代24口<b class="match term0">交换机</b>等技术性器件的安装工作
***
result of  交换

咱俩<b class="match term0">交换</b>一下吧

干事每月经过下属科室都要亲口交代24口<b class="match term0">交换</b>机等技术性器件的安装工作



In [8]:
import jieba
jieba.add_word("好朋友")
for t in analyzer("我的好朋友是李明;我爱北京天安门;IBM和Microsoft; I have a dream. this is intetesting and interested me a lot"):
    print(t.text)


我
朋友
好朋友
是
李明
我
爱
北京
天安
天安门
ibm
microsoft
dream
intetest
interest
me
lot


### 8. 命令行分词

> 使用示例：**python -m jieba news.txt > cut_result.txt**

**命令行选项（翻译）：**

>使用: python -m jieba [options] filename
结巴命令行界面。
固定参数:
  filename              输入文件
可选参数:
  -h, --help            显示此帮助信息并退出
  -d [DELIM], --delimiter [DELIM]
                        使用 DELIM 分隔词语，而不是用默认的' / '。
                        若不指定 DELIM，则使用一个空格分隔。
  -p [DELIM], --pos [DELIM]
                        启用词性标注；如果指定 DELIM，词语和词性之间
                        用它分隔，否则用 _ 分隔
  -D DICT, --dict DICT  使用 DICT 代替默认词典
  -u USER_DICT, --user-dict USER_DICT
                        使用 USER_DICT 作为附加词典，与默认词典或自定义词典配合使用
  -a, --cut-all         全模式分词（不支持词性标注）
  -n, --no-hmm          不使用隐含马尔可夫模型
  -q, --quiet           不输出载入信息到 STDERR
  -V, --version         显示版本信息并退出
如果没有指定文件名，则使用标准输入。>

In [3]:
# help(jieba.analyse.absolute_import)

In [32]:
list(jieba.posseg.cut("试试好不好吧希望好用"))

[pair('试试', 'vn'),
 pair('好不好', 'l'),
 pair('吧', 'y'),
 pair('希望', 'v'),
 pair('好', 'a'),
 pair('用', 'p')]

In [33]:
jieba.lcut("试试好不好吧希望好用")

['试试', '好不好', '吧', '希望', '好用']

In [34]:
jieba.set_dictionary("dict.txt.small.txt")

In [7]:
dir(jieba.posseg)

['CHAR_STATE_TAB_P',
 'POSTokenizer',
 'PROB_EMIT_P',
 'PROB_START_P',
 'PROB_TRANS_P',
 'PY2',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_lcut_internal',
 '_lcut_internal_no_hmm',
 'absolute_import',
 'char_state_tab',
 'char_state_tab_P',
 'cut',
 'default_encoding',
 'dt',
 'emit_P',
 'get_module_res',
 'initialize',
 'iteritems',
 'iterkeys',
 'itervalues',
 'jieba',
 'lcut',
 'load_model',
 'os',
 'pair',
 'pickle',
 'pkg_resources',
 'prob_emit',
 'prob_start',
 'prob_trans',
 're',
 're_eng',
 're_eng1',
 're_han_detail',
 're_han_internal',
 're_num',
 're_skip_detail',
 're_skip_internal',
 'resolve_filename',
 'start_P',
 'strdecode',
 'string_types',
 'sys',
 'text_type',
 'trans_P',
 'unicode_literals',
 'viterbi',
 'xrange']